In [1]:
from contextlib import redirect_stdout
from gurobipy import Model, GRB
import data


# Parametros
C_ij = {(i, j): data.costos_apertura[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo de apertura
Ctikf = {(i, k, f): data.costos_transporte[(f, i, k)] for i in data.ciudades for k in data.regiones for f in data.transportes}  # Costo de transporte
Dk_actual = {k: data.demanda_actual[k] for k in data.regiones}  # Demanda actual
T_k = {k: data.tasas_crecimiento[k] for k in data.regiones}  # Tasa de crecimiento
Dk_t = {(k, t): Dk_actual[k] * (1 + T_k[k])**t for k in data.regiones for t in data.años}  # Demanda futura
Cpp = {j: data.capacidad_planta[j] for j in data.tipos_planta}  # Capacidad de producción
Cv_ij = {(i, j): data.costos_variables[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo variable
Cfij = {(i, j): data.costos_fijos[(i, j)] for i in data.ciudades for j in data.tipos_planta}  # Costo fijo


# Verificacion
'''
print('Costo de transporte: ',C_ij)
print('Demanda actual: ',Dk_actual)
print('Tasa de crecimiento: ',T_k)
print('Demanda futura: ',Dk_t)
print('Capacidad de producción: ',Cpp)
print('Costo variable: ',Cv_ij)
print('Costo fijo: ',Cfij)
'''

# Modelo
model = Model("Optimización de Produccion")


# Variables
X_ij = model.addVars(data.ciudades, data.tipos_planta, vtype=GRB.BINARY, name="X")
Y_ikft = model.addVars(data.ciudades, data.regiones, data.transportes, data.años, vtype=GRB.CONTINUOUS, name="Y")


# Restricciones
for j in data.tipos_planta:
    model.addConstr(sum(X_ij[i, j] for i in data.ciudades) <= 1)

for i in data.ciudades:
    model.addConstr(sum(X_ij[i, j] for j in data.tipos_planta) <= 1)


for i in data.ciudades:
    for j in data.tipos_planta:
        for t in data.años:
            model.addConstr(sum(Y_ikft[i, k, f, t] for k in data.regiones for f in data.transportes) <= X_ij[i, j] * Cpp[j])


D_unmet = model.addVars(data.regiones, data.años, vtype=GRB.CONTINUOUS, name="D_unmet")
for k in data.regiones:
    for t in data.años:
        model.addConstr(sum(Y_ikft[i, k, f, t] for i in data.ciudades for f in data.transportes) + D_unmet[k, t] >= Dk_t[k, t])


# Por lo menos abrir una planta, si no se abre ninguna, la solucion es no abrir nada xd
model.addConstr(
    sum(X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) >= 1,
    "AlMenosUnaPlanta"
)




# Función objetivo (funcion 1 del word)
alpha = 0.1  # Factor de reducción de costos fijos
model.setObjective(
    sum(alpha * (C_ij[i, j] + Cfij[i, j]) * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Cv_ij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Ctikf[i, k, f] * Y_ikft[i, k, f, t] for i in data.ciudades for k in data.regiones for f in data.transportes for t in data.años),
    GRB.MINIMIZE
)



# Función objetivo (funcion 2 del word)
'''
model.setObjective(
    sum(C_ij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum(Cfij[i, j] * X_ij[i, j] for i in data.ciudades for j in data.tipos_planta) +
    sum((Cv_ij[i, j] + Ctikf[i, k, f]) * Y_ikft[i, k, f, t] 
        for i in data.ciudades for j in data.tipos_planta 
        for k in data.regiones for f in data.transportes for t in data.años),
    GRB.MINIMIZE
)
'''



# Optimizando
log = "model.txt"
with open(log, "w", encoding="utf-8") as file:
    with redirect_stdout(file):
        #model.update()
        model.optimize()



# Ver data
for constr in model.getConstrs():
    if constr.Slack < 0:
        print(f"{constr.ConstrName}: Slack = {constr.Slack}")
print("Demanda futura (Dk_t):", Dk_t)
print("Capacidades de producción (Cpp):", Cpp)
for i in data.ciudades:
    for j in data.tipos_planta:
        print(f"X_ij[{i},{j}] = {X_ij[i, j].X}")



# Resultados
with open('results.txt', 'w', encoding='utf-8') as arch:
    arch.write(f'Función objetivo: {model.objVal}\n')
    print(f'Función objetivo: {model.objVal}')

    if model.status == GRB.OPTIMAL:
        for c in data.ciudades:
            for r in data.regiones:
                for t in data.transportes:
                    for an in data.años:
                        if (c, r, t, an) in Y_ikft:
                            valor = Y_ikft[c, r, t, an].X
                            arch.write(f"Y_ikft[{c}, {r}, {t}, {an}] = {valor}\n")
                            print(f"Y_ikft[{c}, {r}, {t}, {an}] = {valor}")



Restricted license - for non-production use only - expires 2026-11-23
R37: Slack = -0.840000000083819
R38: Slack = -0.21440000017173588
R39: Slack = -0.6487040002830327
R40: Slack = -0.9199999999254942
R41: Slack = -0.4224000000394881
R42: Slack = -0.015328000066801906
R43: Slack = -0.7399999999906868
R44: Slack = -0.832399999955669
R45: Slack = -0.4688239998649806
R46: Slack = -0.8000000000465661
R47: Slack = -0.020000000076834112
R48: Slack = -0.07300000009126961
R49: Slack = -0.13999999989755452
R50: Slack = -0.21459999959915876
R51: Slack = -0.12829399900510907
R52: Slack = -0.5
R53: Slack = -0.9499999999534339
R54: Slack = -0.3349999999627471
Demanda futura (Dk_t): {('R1', 1): 1104060.16, ('R1', 2): 1280709.7855999998, ('R1', 3): 1485623.3512959997, ('R2', 1): 1180184.08, ('R2', 2): 1439824.5776, ('R2', 3): 1756585.984672, ('R3', 1): 645184.26, ('R3', 2): 812932.1676, ('R3', 3): 1024294.5311760001, ('R4', 1): 444185.19999999995, ('R4', 2): 510812.9799999999, ('R4', 3): 587434.9269